In [7]:
import LowLevelFEM as FEM
using LowLevelFEM

gmsh.initialize()

In [8]:
# Load the mesh and define material properties
gmsh.open("FEM_kusai.geo")
mat = FEM.material("body", E=2.e5, ν=0.3)

Info    : Reading 'FEM_kusai.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Circle)
Info    : [ 50%] Meshing curve 6 (Circle)
Info    : [ 50%] Meshing curve 7 (Circle)
Info    : [ 60%] Meshing curve 8 (Circle)
Info    : [ 70%] Meshing curve 9 (Circle)
Info    : [ 80%] Meshing curve 10 (Circle)
Info    : [ 90%] Meshing curve 11 (Circle)
Info    : [100%] Meshing curve 12 (Circle)
Info    : Done meshing 1D (Wall 0.0050168s, CPU 0s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 2.94778s, CPU 2.875s)
Info    : 18130 nodes 36273 elements
Info    : Done reading 'FEM_kusai.geo'


("body", 200000.0, 0.3, 7.85e-9)

In [9]:
# Define boundary conditions
problem = FEM.Problem([mat], type="PlaneStress")
supp = FEM.displacementConstraint("supp", ux=0, uy=0)
left_load = FEM.load("left", fy=-1);  # Assuming q = -1 N/mm^2 for left side
right_load = FEM.load("right", fy=1); # Assuming q = 1 N/mm^2 for right side

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 141)
Info    : Mapping does not contain a node tag (6) - incrementing after last provided tag (18125)


In [10]:
# Assemble stiffness matrix and load vector
K = FEM.stiffnessMatrix(problem)
f = FEM.loadVector(problem, [left_load, right_load])

36250-element Vector{Float64}:
  0.0
 -0.5
  0.0
  0.0
  0.0
  0.0
  0.0
 -1.0
  0.0
 -1.0
  0.0
 -1.0
  0.0
  ⋮
  0.0
  0.0
  0.0
  1.0
  0.0
  0.0
  0.0
  0.0
  0.0
  1.0
  0.0
  0.5

In [11]:
# Apply boundary conditions
FEM.applyBoundaryConditions!(problem, K, f, [supp])

LoadError: BoundsError

In [12]:
# Solve for displacement and stress
q = FEM.solveDisplacement(K, f)
S = FEM.solveStress(problem, q)

LowLevelFEM.StressField([[-57.34621823489011; -15.992384690504807; … ; 0.0; 0.0;;], [-15.761826064560438; -2.3475060096153846; … ; 0.0; 0.0;;], [-8.383950034340659; 0.0; … ; 0.0; 0.0;;], [-23.642739096840657; -1.7606295072115383; … ; 0.0; 0.0;;], [-23.81041809752747; 4.695012019230769; … ; 0.0; 0.0;;], [6.87483902815934; -9.390024038461538; … ; 0.0; 0.0;;], [51.14209520947802; -7.62939453125; … ; 0.0; 0.0;;], [-16.36933494400192; 7.869532475104698; … ; 0.0; 0.0;;], [-26.701570867182134; -17.32833110369169; … ; 0.0; 0.0;;], [26.7448006095467; -2.3475060096153846; … ; 0.0; 0.0;;]  …  [18.69096860780821; 8.252950815054087; … ; 0.0; 0.0;;], [-6.707160027472527; -9.390024038461538; … ; 0.0; 0.0;;], [77.71921681833791; 0.0; … ; 0.0; 0.0;;], [-17.438616071428573; 0.0; … ; 0.0; 0.0;;], [3.3535800137362637; 21.12755408653846; … ; 0.0; 0.0;;], [27.49935611263736; -9.390024038461538; … ; 0.0; 0.0;;], [4.695012019230769; -8.216271033653847; … ; 0.0; 0.0;;], [-74.70099480597528; 11.737530048076923;

In [13]:
# Visualize results
u = FEM.showDoFResults(problem, q, "uvec", name="uvec", visible=false)
ux = FEM.showDoFResults(problem, q, "ux", name="ux", visible=false)
uy = FEM.showDoFResults(problem, q, "uy", name="uy", visible=false)

"uvec..ok"

"ux..ok"

"uy..ok"

3

In [14]:
s = FEM.showStressResults(problem, S, "s", name="σ", visible=true, smooth=true)
sx = FEM.showStressResults(problem, S, "sx", name="σx", visible=false, smooth=true)
sy = FEM.showStressResults(problem, S, "sy", name="σy", visible=false, smooth=true)
sxy = FEM.showStressResults(problem, S, "sxy", name="τxy", visible=false, smooth=true)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


"s..ok"

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


"sx..ok"

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


"sy..ok"

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


"sxy..ok"

7

In [15]:
FEM.plotOnPath(problem, "path", sx, 100, name="σx", visible=false)
FEM.plotOnPath(problem, "path", sxy, 100, name="τxy", visible=false)
FEM.plotOnPath(problem, "path", ux, 100, name="ux", visible=false)

10

In [ ]:
# Run the visualization
gmsh.fltk.run()
gmsh.finalize()